In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [4]:
len(data)

174643

In [5]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [8]:
# hard-coded awards
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [9]:
import re

In [10]:
def reduce(line):
    # remove words "performance", "in", "a", "an", "made", "for", "by", and all punctuations
    return re.sub(r'\bperformance\b|\bin\b|\ba\b|\ban\b|\bmade\b|\bfor\b|\bby\b|[^\w\s]', ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
for award in awards:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['best', 'motion', 'picture', 'drama']
['best', 'motion', 'picture', 'musical', 'or', 'comedy']
['best', 'actress', 'motion', 'picture', 'drama']
['best', 'actor', 'motion', 'picture', 'drama']
['best', 'actress', 'motion', 'picture', 'musical', 'or', 'comedy']
['best', 'actor', 'motion', 'picture', 'musical', 'or', 'comedy']
['best', 'actress', 'supporting', 'role', 'any', 'motion', 'picture']
['best', 'actor', 'supporting', 'role', 'any', 'motion', 'picture']
['best', 'director', 'motion', 'picture']
['best', 'screenplay', 'motion', 'picture']
['best', 'motion', 'picture', 'animated']
['best', 'motion', 'picture', 'foreign', 'language']
['best', 'original', 'score', 'motion', 'picture']
['best', 'original', 'song', 'motion', 'picture']
['best', 'television', 'series', 'drama']
['best', 'television', 'series', 'musical', 'or', 'comedy']
['best', 'television', 'limited', 'series', 'or', 'motion', 'picture']
['best', 'actress', 'limited', 'series', 'or', 'motion', 'picture', 'television

In [11]:
def generate_award_num_keywords_map(awards):
    award_num_keywords_map = {}
    
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            num_words = len(award) - 1 - award.index('or') # number of words from "or" to the end of award name
            # test
            # print(len(award)-1, award.index('or'), num_words)
            award.remove('or')
            award_num_keywords_map[i] = len(award) - 1 - num_words # do not count "best" and words from "or" to the end of award name
        else:
            award_num_keywords_map[i] = len(award) - 1 # do not count "best"
    return award_num_keywords_map

In [12]:
award_num_keywords_map = generate_award_num_keywords_map(awards)

sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

[(0, 3),
 (1, 3),
 (2, 4),
 (3, 4),
 (4, 4),
 (5, 4),
 (6, 6),
 (7, 6),
 (8, 3),
 (9, 3),
 (10, 3),
 (11, 4),
 (12, 4),
 (13, 4),
 (14, 3),
 (15, 3),
 (16, 3),
 (17, 3),
 (18, 3),
 (19, 4),
 (20, 4),
 (21, 4),
 (22, 4),
 (23, 5),
 (24, 5),
 (25, 3)]

In [13]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
        
    return re.sub(pattern, ' ', line)

In [14]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [15]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [16]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

# test
cleanse("Hugh Jackman is so awesome!!! #goldenglobes")

'Hugh Jackman is so awesome    #goldenglobes'

In [17]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [18]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
    return tags

In [24]:
def find_presenter(award_index, verbose=False):
    pattern = re.compile(r'presenter|present', re.IGNORECASE)
    entity_dict = {}
    
    print('Predicting for:', awards[award_index])
    
    num = 0
    for tweet in data:
        # remove_retweet_prefix 
        line = remove_retweet_prefix(tweet)
        # remove hashtag
        line = remove_hashtag(line)
        # remove @...
        line = remove_at(line)
        # remove punctuations
        line = cleanse(line)
        
        award = awards_reduced[award_index]
        num_keywords_to_match = award_num_keywords_map[award_index]
        
        # test for 'best actress in a motion picture comedy or musical'
#         if len(set(award).intersection(set(line.split()))) > 5:
        
#         if len(set(award).intersection(set(line.split()))) > num_keywords_to_match:
        # test
        if len(set(award).intersection(set(line.split()))) > 1: # word overlap number needs to further adjusted
            
            match = re.findall(pattern, line.lower())
            if match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(tweet)
                        print(tags)
                        print()

                for entity in tags.keys():
                    if len(entity) > 1:
                        if entity not in entity_dict:
                            entity_dict[entity] = 1
                        else:
                            entity_dict[entity] += 1
                num += 1
            
    print('num of matches:', num)
    return entity_dict

# Generate Predictions for Presenter

In [25]:
for key in award_num_keywords_map.keys():
    print(sorted(find_presenter(key, verbose=False).items(), key=lambda pair: pair[1], reverse=True)[:100])
    print()

Predicting for: Best Motion Picture - Drama
num of matches: 18
[('Lincoln', 10), ('Julia Roberts', 7), ('Golden Globes', 3), ('Clinton', 2), ('Lancome Ambassadress', 2), ('nine', 1), ('Makes', 1), ('night   BECAUSE WHERE', 1), ('Roberts', 1)]

Predicting for: Best Motion Picture - Musical or Comedy
num of matches: 21
[('Lincoln', 10), ('Julia Roberts', 7), ('Comedians', 3), ('Kristen Wiig', 3), ('Will Ferrell', 3), ('Clinton', 2), ('Lancome Ambassadress', 2), ('Dennis Quaid', 1), ('Kerry Washington', 1), ('nine', 1), ('Makes', 1), ('Bill Clinton', 1), ('Lincon', 1), ('Jennifer Lawrence', 1), ('Jay Leno', 1), ('Jimmy Fallon', 1), ('night   BECAUSE WHERE', 1), ('Roberts', 1)]

Predicting for: Best Performance by an Actress in a Motion Picture - Drama
num of matches: 21
[('Lincoln', 10), ('Julia Roberts', 7), ('Golden Globes', 3), ('Clinton', 2), ('Lancome Ambassadress', 2), ('Dennis Quaid', 1), ('Kerry Washington', 1), ('nine', 1), ('Makes', 1), ('Bill Clinton', 1), ('Lincon', 1), ('Jenn

num of matches: 30
[('Maggie Smith', 21), ('Downton Abbey', 21), ('Comedians', 3), ('Kristen Wiig', 3), ('Will Ferrell', 3), ('Dennis Quaid', 1), ('Kerry Washington', 1), ('George W', 1), ('Bush', 1), ('Bill Clinton', 1), ('Lincon', 1), ('Jennifer Lawrence', 1), ('Jay Leno', 1), ('Jimmy Fallon', 1)]

Predicting for: Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television
num of matches: 46
[('Maggie Smith', 21), ('Downton Abbey', 21), ('Lincoln', 10), ('Julia Roberts', 7), ('Lindsay Lohan', 5), ('first', 5), ('Clinton', 2), ('Lancome Ambassadress', 2), ('Dennis Quaid', 1), ('Kerry Washington', 1), ('nine', 1), ('Makes', 1), ('George W', 1), ('Bush', 1), ('Bill Clinton', 1), ('Lincon', 1), ('Jennifer Lawrence', 1), ('Jay Leno', 1), ('Jimmy Fallon', 1), ('night   BECAUSE WHERE', 1), ('Roberts', 1)]

Predicting for: Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Televisi

In [18]:
def weighted_freq(element):
    return entity_dict[element] * len(element)

e = entity_dict.copy()
for cluster in names_clusters_reduced:
    # select the longest name
    selected_name = max(cluster, key=weighted_freq)
    cluster.remove(selected_name)
    for name in cluster:
        # if not deleted in previous cases
        if name in e and selected_name in e:
            e[selected_name] += e[name]
            del e[name]